# Segmentation in multiple channels: big structure from mixed channels, small structures from (green - red)

Goal:
- Split channels of the input image.
- a) Segment the big structure from a mix of all channels.
- b) Segment the small structures from the difference image (green channel minus red channel).
- Fill holes in both binary masks.
- Create outlines and overlay them on the original image: big structure in yellow, small structures in green.

Notes:
- Intermediate results are saved to disk in the `results/` folder.
- If image download fails, a synthetic image is generated so the notebook runs end-to-end.

In [1]:
# Imports and setup
import os
import numpy as np
from skimage.io import imread, imsave
from skimage import exposure, util
from skimage.filters import gaussian, threshold_otsu
from skimage.morphology import remove_small_objects, remove_small_holes, white_tophat, disk
from skimage.measure import find_contours
from scipy import ndimage as ndi
import matplotlib.pyplot as plt

# Optional libraries from the requested toolset (used if available)
try:
    import napari_simpleitk_image_processing as nsitk  # noqa: F401
    HAS_NSITK = True
except Exception:
    HAS_NSITK = False
try:
    import napari_segment_blobs_and_things_with_membranes as nsbatwm  # noqa: F401
    HAS_NSBATWM = True
except Exception:
    HAS_NSBATWM = False

os.makedirs('results', exist_ok=True)

def to_uint8(image):
    """Rescale image to [0,255] and convert to uint8."""
    img = exposure.rescale_intensity(image, in_range='image', out_range=(0, 1))
    return util.img_as_ubyte(img)

def save_png(path, image):
    imsave(path, to_uint8(image))

/opt/hostedtoolcache/Python/3.13.7/x64/lib/python3.13/site-packages/napari_tools_menu/__init__.py:10: UserWarning: Importing QT failed; now introducing dummy definitions of QMenu class and register_function decorator.
  warnings.warn("Importing QT failed; now introducing dummy definitions of QMenu class and register_function decorator.")
/opt/hostedtoolcache/Python/3.13.7/x64/lib/python3.13/site-packages/napari_skimage_regionprops/_table.py:7: UserWarning: No Qt bindings could be found
  warnings.warn(str(e))


Load the RGB image from GitHub. If it cannot be downloaded, create a synthetic fallback image that resembles a long greenish tube with red puncta on black background. Save the original to disk for reference.

In [2]:
url = 'https://github.com/user-attachments/assets/a1501925-24ca-48ed-a222-b5d8a4bb0ea4'

def generate_synthetic_image(h=512, w=384, seed=0):
    rng = np.random.default_rng(seed)
    img = np.zeros((h, w, 3), dtype=np.float32)
    # Create a vertical tube in green channel with a bifurcation
    rr = np.arange(h)
    center_x = w//2
    radius = 20
    Y, X = np.meshgrid(rr, np.arange(w), indexing='ij')
    tube = (np.abs(X - center_x) < radius).astype(float)
    # Bifurcation near top
    yb = int(h*0.2)
    branch = ((X - (center_x - (rr - yb)*0.15))**2 + (Y - rr)**2 < (radius*0.6)**2) & (Y < yb)
    branch |= ((X - (center_x + (rr - yb)*0.15))**2 + (Y - rr)**2 < (radius*0.6)**2) & (Y < yb)
    g = np.clip(0.6*tube + 0.6*branch.astype(float), 0, 1)
    # Red puncta along tube edges
    r = np.zeros_like(g)
    n_spots = 200
    ys = rng.integers(low=int(h*0.05), high=int(h*0.95), size=n_spots)
    xs = rng.normal(loc=center_x + rng.choice([-1,1], size=n_spots)*radius*0.9, scale=4, size=n_spots).astype(int)
    xs = np.clip(xs, 0, w-1)
    for y, x in zip(ys, xs):
        rr0 = slice(max(0, y-2), min(h, y+3))
        cc0 = slice(max(0, x-2), min(w, x+3))
        r[rr0, cc0] = 1.0
    # Blur slightly to resemble real data
    g = gaussian(g, sigma=1)
    r = gaussian(r, sigma=0.7)
    # Compose RGB (add faint grayscale mix)
    b = g*0.2
    rgb = np.dstack([r, g, b])
    return exposure.rescale_intensity(rgb, out_range=(0,1))

try:
    rgb = imread(url)
    # If image comes as RGBA, drop alpha
    if rgb.ndim == 3 and rgb.shape[2] == 4:
        rgb = rgb[:, :, :3]
    rgb = exposure.rescale_intensity(rgb, out_range=(0,1)).astype(np.float32)
except Exception:
    rgb = generate_synthetic_image()

save_png('results/00_original.png', rgb)

Split channels and compute derived images:
- Mixed-channel grayscale for the big structure (mean of R,G,B).
- Difference image for small structures: green minus red, rescaled to [0,1]. Save these images for inspection.

In [3]:
# Split channels
R = rgb[..., 0].astype(np.float32)
G = rgb[..., 1].astype(np.float32)
B = rgb[..., 2].astype(np.float32)

# Mixed-channel grayscale (mean)
gray_mix = (R + G + B) / 3.0

# Difference image: green minus red
diff_gr = G - R
diff_gr = exposure.rescale_intensity(diff_gr, in_range='image', out_range=(0,1)).astype(np.float32)

# Save intermediates
save_png('results/01_channel_R.png', R)
save_png('results/02_channel_G.png', G)
save_png('results/03_channel_B.png', B)
save_png('results/04_gray_mix.png', gray_mix)
save_png('results/05_diff_G_minus_R.png', diff_gr)

Segment the big structure from the mixed-channel image:
- Gaussian smoothing
- Otsu threshold
- Remove small objects and fill holes
- Save the binary mask

In [4]:
# Big structure segmentation on gray_mix
g_blur = gaussian(gray_mix, sigma=2.0)
thr_big = threshold_otsu(g_blur)
big_bin = g_blur > thr_big
big_bin = remove_small_objects(big_bin, min_size=1000)
big_bin = remove_small_holes(big_bin, area_threshold=2000)
big_bin = ndi.binary_fill_holes(big_bin)

save_png('results/06_big_structure_mask.png', big_bin.astype(np.float32))

Segment small structures from the (green − red) difference image:
- White top-hat to enhance small bright features
- Gaussian smoothing and Otsu threshold
- Remove small objects and fill holes
- Save the binary mask

In [5]:
# Small structures segmentation on (G - R)
tophat = white_tophat(diff_gr, footprint=disk(3))
s_blur = gaussian(tophat, sigma=1.0)
thr_small = threshold_otsu(s_blur)
small_bin = s_blur > thr_small
small_bin = remove_small_objects(small_bin, min_size=20)
small_bin = ndi.binary_fill_holes(small_bin)

save_png('results/07_small_structures_mask.png', small_bin.astype(np.float32))

Create outlines from both masks and overlay them on the original image using matplotlib:
- Big structure outline in yellow
- Small structures outlines in green
Save the composite figure to disk.

In [6]:
# Find contours for overlays
contours_big = find_contours(big_bin.astype(float), level=0.5)
contours_small = find_contours(small_bin.astype(float), level=0.5)

# Overlay and save
fig, ax = plt.subplots(figsize=(6, 8))
ax.imshow(to_uint8(rgb))
for c in contours_big:
    ax.plot(c[:, 1], c[:, 0], color='yellow', linewidth=2, alpha=0.9)
for c in contours_small:
    ax.plot(c[:, 1], c[:, 0], color='lime', linewidth=1.5, alpha=0.9)
ax.set_axis_off()
plt.tight_layout()
fig.savefig('results/08_overlay_contours.png', dpi=200, bbox_inches='tight', pad_inches=0)
plt.close(fig)

Save arrays for later reuse (optional): masks and processed images as .npy files.

In [7]:
np.save('results/big_structure_mask.npy', big_bin.astype(np.uint8))
np.save('results/small_structures_mask.npy', small_bin.astype(np.uint8))
np.save('results/gray_mix.npy', gray_mix.astype(np.float32))
np.save('results/diff_G_minus_R.npy', diff_gr.astype(np.float32))